# Climate Watch API data  
Prototype to extract data from the [Climate Watch API](https://www.climatewatchdata.org) to feed the "International Status" widget with latest data.  
The indicators that we seek are:  
- NDC document link  
- Date of most recent NDC  
- NDC target (percentage)  
- Base year  
- Base year emissions (actual)  
- Target Reference year  
- Target year emissions (projected)  
- Mitigation only, Adaptation only, or both  

In [130]:
import pandas as pd
import requests

## Set up API call

In [131]:
api_url = "https://www.climatewatchdata.org/api/v1/"

data_endpoint = ""
indicator_var = ""

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators= {indicator_var}'

print(api_call)


https://www.climatewatchdata.org/api/v1/


## Indicator 1: Document link + Submission date

In [132]:
data_endpoint = "ndcs/countries_documents"
indicator_var = ""

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs/countries_documents


In [133]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['data'].keys())

In [134]:
test = pd.DataFrame(json_data['data'][locations_keys[0]])
test

,id,ordering,slug,long_name,description,is_ndc,submission_date,url
0,3034.0,1,indc,Intended Nationally Determined Contribution (I...,INDC Submission,True,2015-10-13,http://www4.unfccc.int/sites/submissions/INDC/...
1,3035.0,2,first_ndc,First Nationally Determined Contribution (NDC),First NDC Submission,True,2016-11-23,https://unfccc.int/sites/default/files/NDC/202...
2,3039.0,6,pledges,Pre-2020 Pledges,Pre-2020 Pledges,False,2013-06-14,http://unfccc.int/files/meetings/cop_15/copenh...
3,NaN,7,sectoral,Sectoral Laws or Policies,Sectoral Laws or Policies,False,None,NaN


In [135]:
ind_1 = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['data'][k])
    t['location'] = k
    ind_1 = ind_1.append(t)

ind_1 = ind_1[['location', 'ordering', 'slug', 'is_ndc','submission_date', 'url']]
ind_1 = ind_1[ind_1['is_ndc']].reset_index(drop=True)
ind_1.rename(columns={'slug': 'document_slug'}, inplace=True)
ind_1

,location,ordering,document_slug,is_ndc,submission_date,url
0,AFG,1.0,indc,True,2015-10-13,http://www4.unfccc.int/sites/submissions/INDC/...
1,AFG,2.0,first_ndc,True,2016-11-23,https://unfccc.int/sites/default/files/NDC/202...
2,ALB,1.0,indc,True,2015-09-24,http://www4.unfccc.int/sites/submissions/INDC/...
3,ALB,2.0,first_ndc,True,2016-09-21,https://unfccc.int/sites/default/files/NDC/202...
4,ALB,3.0,revised_first_ndc,True,2021-10-12,https://unfccc.int/sites/default/files/NDC/202...
...,...,...,...,...,...,...
555,ZMB,2.0,first_ndc,True,2016-12-09,https://unfccc.int/sites/default/files/NDC/202...
556,ZMB,3.0,revised_first_ndc,True,2021-07-30,https://unfccc.int/sites/default/files/NDC/202...
557,ZWE,1.0,indc,True,2015-09-30,http://www4.unfccc.int/sites/submissions/INDC/...
558,ZWE,2.0,first_ndc,True,2017-08-07,https://unfccc.int/sites/default/files/NDC/202...


In [136]:
len(ind_1['location'].unique())

197

## Indicator 2a: reduction compared to base year (percentage)

In [137]:
data_endpoint = "ndcs"
indicator_var = "M_TarA2"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=M_TarA2


In [138]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [139]:
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[0]])
test

,value,document_slug,group_index
0,-40%,revised_first_ndc,None


In [140]:
ind_2a = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_2a = ind_2a.append(t)

ind_2a = ind_2a[['location', 'value', 'document_slug']]
ind_2a = ind_2a.rename(columns={'value': 'pct_reduction_base_year'})
ind_2a

,location,pct_reduction_base_year,document_slug
0,ARM,-40%,revised_first_ndc
0,AUS,-43%,revised_first_ndc
1,AUS,-26% to -28%,first_ndc
2,AUS,-26% to -28%,indc
3,AUS,26-28%,None
...,...,...,...
0,ZAF,Not Specified,revised_first_ndc
0,ZMB,-47%,revised_first_ndc
1,ZMB,-47%,first_ndc
2,ZMB,-47%,indc


## Indicator 2b: reduction compared to baseline (percentage)

In [141]:
data_endpoint = "ndcs"
indicator_var = "M_TarB1"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=M_TarB1


In [142]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [143]:
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[0]])
test

,value,document_slug,group_index
0,-14%,first_ndc,None
1,-14%,indc,None


In [144]:
ind_2b = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_2b = ind_2b.append(t)

ind_2b = ind_2b[['location', 'value', 'document_slug']]
ind_2b = ind_2b.rename(columns={'value': 'pct_reduction_baseline'})
ind_2b

,location,pct_reduction_baseline,document_slug
0,AFG,-14%,first_ndc
1,AFG,-14%,indc
0,AGO,-24%,revised_first_ndc
1,AGO,-50%,first_ndc
2,AGO,-50%,indc
...,...,...,...
1,VNM,-25%,first_ndc
2,VNM,-25%,indc
3,VNM,Unconditional: -9%,None
0,YEM,-14%,indc


## Indicator 3: Base year

In [145]:
data_endpoint = "ndcs"
indicator_var = "M_TarA4"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=M_TarA4


In [146]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [147]:
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[0]])
test['location'] = locations_keys[0]
test

,value,document_slug,group_index,location
0,1990,revised_first_ndc,None,ARM


In [148]:
ind_3 = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_3 = ind_3.append(t)

ind_3 = ind_3[['location', 'value', 'document_slug']]
ind_3.rename(columns={'value': 'base_year'}, inplace=True)
ind_3

,location,base_year,document_slug
0,ARM,1990,revised_first_ndc
0,AUS,2005,revised_first_ndc
1,AUS,2005,first_ndc
2,AUS,2005,indc
3,AUS,2005,None
...,...,...,...
0,WSM,2007,second_ndc
0,ZMB,2010,revised_first_ndc
1,ZMB,2010,first_ndc
2,ZMB,2010,indc


## Indicator 4: Target year

In [149]:
data_endpoint = "ndcs"
indicator_var = "M_TarYr"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

https://www.climatewatchdata.org/api/v1/ndcs?indicators=M_TarYr


In [150]:
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[0]])
test['location'] = locations_keys[0]
test

,value,document_slug,group_index,location
0,2030,first_ndc,None,AFG
1,2030,indc,None,AFG


In [151]:
ind_4 = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_4 = ind_4.append(t)

ind_4 = ind_4[['location', 'value', 'document_slug']]
ind_4.rename(columns={'value': 'target_year'}, inplace=True)
ind_4

,location,target_year,document_slug
0,AFG,2030,first_ndc
1,AFG,2030,indc
0,AGO,2025,revised_first_ndc
1,AGO,2030,first_ndc
2,AGO,2030,indc
...,...,...,...
2,ZMB,2030,indc
3,ZMB,2030,None
0,ZWE,2030,revised_first_ndc
1,ZWE,2030,first_ndc


## Indicator 5a: Emission reduction from base year (absolute)

In [152]:
data_endpoint = "ndcs"
indicator_var = "M_TarA1"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=M_TarA1


In [153]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [154]:
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[2]])
test['location'] = locations_keys[2]
test

,value,document_slug,group_index,location
0,Not Specified,revised_first_ndc,None,AUS
1,Not Specified,None,None,AUS


In [155]:
ind_5a = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_5a = ind_5a.append(t)

ind_5a = ind_5a[['location', 'value', 'document_slug']]
ind_5a = ind_5a.rename(columns={'value': 'reduction_base_year'})
ind_5a

,location,reduction_base_year,document_slug
0,ARG,Not Specified,second_ndc
1,ARG,369,first_ndc
2,ARG,An economy-wide unconditional cap net emission...,None
0,ARM,Not Specified,revised_first_ndc
0,AUS,Not Specified,revised_first_ndc
...,...,...,...
0,ZAF,Not Specified,revised_first_ndc
1,ZAF,398 and 614 MtCO2e by 2025 and 2030,first_ndc
2,ZAF,398 and 614 MtCO2e by 2025 and 2030,indc
0,ZMB,n/a,first_ndc


## Indicator 5b: Emission reduction from baseline (absolute)

In [156]:
data_endpoint = "ndcs"
indicator_var = "M_TarA5"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=M_TarA5


In [157]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [158]:
loc_n = 4
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[loc_n]])
test['location'] = locations_keys[loc_n]
test

,value,document_slug,group_index,location
0,-5.4983,first_ndc,None,CAF
1,-5.4983,indc,None,CAF


In [159]:
ind_5b = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_5b = ind_5b.append(t)

ind_5b = ind_5b[['location', 'value', 'document_slug']]
ind_5b = ind_5b.rename(columns={'value': 'reduction_baseline'})
ind_5b

,location,reduction_baseline,document_slug
0,AND,0.194,first_ndc
1,AND,0.194,indc
2,AND,0.194,None
0,BDI,-14.897,first_ndc
1,BDI,-14.897,indc
0,BEN,-49.49 MtCO2e (cumulative 2021-2030),first_ndc
0,BFA,13.7663,first_ndc
1,BFA,13.7663,indc
0,CAF,-5.4983,first_ndc
1,CAF,-5.4983,indc


## Indicator 6: NDC blurb

In [160]:
data_endpoint = "ndcs"
indicator_var = "pledge_content"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=pledge_content


In [161]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [162]:
loc_n = 1
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[loc_n]])
test['location'] = locations_keys[loc_n]
test

,value,document_slug,group_index,location
0,Argentina communicated that it is currently de...,pledges,None,ARG


In [163]:
ind_6 = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_6 = ind_6.append(t)

ind_6 = ind_6[['location', 'value', 'document_slug']]
ind_6 = ind_6.rename(columns={'value': 'ndc_blurb'})
ind_6

,location,ndc_blurb,document_slug
0,AFG,Afghanistan communicated that its NAMAs would ...,pledges
0,ARG,Argentina communicated that it is currently de...,pledges
0,ARM,Armenia communicated the following NAMAs: <br ...,pledges
0,ATG,Antigua and Barbuda communicated that it would...,pledges
0,AUS,Australia will reduce its greenhouse gas (GHG)...,pledges
...,...,...,...
0,TJK,Tajikistan communicated the following NAMAs:<b...,pledges
0,TUN,Tunisia communicated the following NAMAs:<br /...,pledges
0,UKR,Ukraine communicated a target of a 20 per cent...,pledges
0,USA,The United States communicated a target in the...,pledges


## Indicator 7: pledge type

In [164]:
data_endpoint = "ndcs"
indicator_var = "mitigation_contribution_type"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=mitigation_contribution_type


In [165]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [166]:
loc_n = 1
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[loc_n]])
test['location'] = locations_keys[loc_n]
test

,value,label_id,document_slug,group_index,location
0,GHG target,529627,revised_first_ndc,None,AGO
1,GHG target,529627,first_ndc,None,AGO
2,GHG target,529627,indc,None,AGO


In [167]:
ind_7 = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_7 = ind_7.append(t)

ind_7 = ind_7[['location', 'value', 'document_slug']]
ind_7 = ind_7.rename(columns={'value': 'pledge_type'})
ind_7

,location,pledge_type,document_slug
0,AFG,GHG target,first_ndc
1,AFG,GHG target,indc
0,AGO,GHG target,revised_first_ndc
1,AGO,GHG target,first_ndc
2,AGO,GHG target,indc
...,...,...,...
2,ZMB,GHG target,indc
3,ZMB,GHG target,None
0,ZWE,GHG target,revised_first_ndc
1,ZWE,GHG target,first_ndc


## Indicator 8: adaptation

In [168]:
data_endpoint = "ndcs"
indicator_var = "adaptation"

if (indicator_var == ""):
    api_call = api_url+data_endpoint
else:
    api_call = f'{api_url}{data_endpoint}?indicators={indicator_var}'

print(api_call)

https://www.climatewatchdata.org/api/v1/ndcs?indicators=adaptation


In [169]:
c = requests.get(api_call)
json_data = c.json()
locations_keys = list(json_data['indicators'][0]['locations'].keys())

In [170]:
loc_n = 1
test = pd.DataFrame(json_data['indicators'][0]['locations'][locations_keys[loc_n]])
test['location'] = locations_keys[loc_n]
test

,value,label_id,document_slug,group_index,location
0,Yes,529663,revised_first_ndc,None,AGO
1,Yes,529663,first_ndc,None,AGO
2,Yes,529663,indc,None,AGO


In [171]:
ind_8 = pd.DataFrame()
for k in locations_keys:
    t = pd.DataFrame(json_data['indicators'][0]['locations'][k])
    t['location'] = k
    ind_8 = ind_8.append(t)

ind_8 = ind_8[['location', 'value', 'document_slug']]
ind_8 = ind_8.rename(columns={'value': 'adaptation'})
ind_8

,location,adaptation,document_slug
0,AFG,Yes,first_ndc
1,AFG,Yes,indc
0,AGO,Yes,revised_first_ndc
1,AGO,Yes,first_ndc
2,AGO,Yes,indc
...,...,...,...
2,ZMB,Yes,indc
3,ZMB,Yes,None
0,ZWE,Yes,revised_first_ndc
1,ZWE,Yes,first_ndc


## Concat all

In [172]:
df_concat = pd.merge(ind_1, ind_2a, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_2b, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_3, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_4, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_5a, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_5b, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_6, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_7, on=['location', 'document_slug'], how='left')
df_concat = pd.merge(df_concat, ind_8, on=['location', 'document_slug'], how='left')
df_concat

,location,ordering,document_slug,is_ndc,submission_date,url,pct_reduction_base_year,pct_reduction_baseline,base_year,target_year,reduction_base_year,reduction_baseline,ndc_blurb,pledge_type,adaptation
0,AFG,1.0,indc,True,2015-10-13,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,-14%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
1,AFG,2.0,first_ndc,True,2016-11-23,https://unfccc.int/sites/default/files/NDC/202...,NaN,-14%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
2,ALB,1.0,indc,True,2015-09-24,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,-12%,NaN,2030,NaN,NaN,NaN,GHG target,No
3,ALB,2.0,first_ndc,True,2016-09-21,https://unfccc.int/sites/default/files/NDC/202...,NaN,-12%,NaN,2030,NaN,NaN,NaN,GHG target,No
4,ALB,3.0,revised_first_ndc,True,2021-10-12,https://unfccc.int/sites/default/files/NDC/202...,NaN,-20.90%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,ZMB,2.0,first_ndc,True,2016-12-09,https://unfccc.int/sites/default/files/NDC/202...,-47%,NaN,2010,2030,n/a,38,NaN,GHG target,Yes
556,ZMB,3.0,revised_first_ndc,True,2021-07-30,https://unfccc.int/sites/default/files/NDC/202...,-47%,NaN,2010,2030,NaN,NaN,NaN,GHG target,Yes
557,ZWE,1.0,indc,True,2015-09-30,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,NaN,NaN,2030,NaN,NaN,NaN,GHG target,Yes
558,ZWE,2.0,first_ndc,True,2017-08-07,https://unfccc.int/sites/default/files/NDC/202...,NaN,NaN,NaN,2030,NaN,NaN,NaN,GHG target,Yes


In [175]:
df_concat[df_concat['location'] == 'AGO']

,location,ordering,document_slug,is_ndc,submission_date,url,pct_reduction_base_year,pct_reduction_baseline,base_year,target_year,reduction_base_year,reduction_baseline,ndc_blurb,pledge_type,adaptation
10,AGO,1.0,indc,True,2015-11-29,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,-50%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
11,AGO,2.0,first_ndc,True,2020-11-15,https://unfccc.int/sites/default/files/NDC/202...,NaN,-50%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
12,AGO,3.0,revised_first_ndc,True,2021-05-30,https://unfccc.int/sites/default/files/NDC/202...,NaN,-24%,NaN,2025,NaN,NaN,NaN,GHG target,Yes


In [176]:
df_concat[(df_concat['pct_reduction_base_year'].isna()) & (df_concat['pct_reduction_baseline'].isna())]

,location,ordering,document_slug,is_ndc,submission_date,url,pct_reduction_base_year,pct_reduction_baseline,base_year,target_year,reduction_base_year,reduction_baseline,ndc_blurb,pledge_type,adaptation
13,ATG,1.0,indc,True,2015-10-19,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,NaN,NaN,2030,NaN,NaN,NaN,Non-GHG target and actions,Yes
14,ATG,2.0,first_ndc,True,2016-09-21,https://unfccc.int/sites/default/files/NDC/202...,NaN,NaN,NaN,2030,NaN,NaN,NaN,Non-GHG target and actions,Yes
15,ATG,3.0,revised_first_ndc,True,2021-09-02,https://unfccc.int/sites/default/files/NDC/202...,NaN,NaN,NaN,2030,NaN,NaN,NaN,Non-GHG target and actions,Yes
16,ARG,1.0,indc,True,2015-10-01,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GHG target,Yes
17,ARG,2.0,first_ndc,True,2016-11-17,https://unfccc.int/sites/default/files/1711201...,NaN,NaN,NaN,2030,369,NaN,NaN,GHG target,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,VUT,2.0,first_ndc,True,2016-09-21,https://unfccc.int/sites/default/files/NDC/202...,NaN,NaN,NaN,2030,NaN,NaN,NaN,Non-GHG target only,Yes
546,VUT,3.0,revised_first_ndc,True,2022-08-09,https://unfccc.int/sites/default/files/NDC/202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-GHG target and actions,Yes
547,VEN,1.0,indc,True,2015-12-15,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GHG target,Yes
557,ZWE,1.0,indc,True,2015-09-30,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,NaN,NaN,2030,NaN,NaN,NaN,GHG target,Yes


In [174]:
df_concat['pledge_type'].value_counts(dropna=False)

GHG target                       400
GHG target and non-GHG target     63
Actions only                      54
Non-GHG target and actions        32
Non-GHG target only               11
Name: pledge_type, dtype: int64

## Generate widget data table

In [183]:
df_final = pd.DataFrame()

for l in df_concat['location'].unique():
    t = df_concat[df_concat['location'] == l]
    t = t.sort_values(by=['ordering'], ascending=False)
    t = t.drop_duplicates(subset=['location'], keep='first')
    df_final = df_final.append(t)

df_final.reset_index(drop=True, inplace=True)
df_final

,location,ordering,document_slug,is_ndc,submission_date,url,pct_reduction_base_year,pct_reduction_baseline,base_year,target_year,reduction_base_year,reduction_baseline,ndc_blurb,pledge_type,adaptation
0,AFG,2.0,first_ndc,True,2016-11-23,https://unfccc.int/sites/default/files/NDC/202...,NaN,-14%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
1,ALB,3.0,revised_first_ndc,True,2021-10-12,https://unfccc.int/sites/default/files/NDC/202...,NaN,-20.90%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
2,DZA,2.0,first_ndc,True,2016-10-20,https://unfccc.int/sites/default/files/NDC/202...,NaN,-7%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
3,AND,3.0,revised_first_ndc,True,2022-11-08,https://unfccc.int/sites/default/files/NDC/202...,NaN,-55%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
4,AGO,3.0,revised_first_ndc,True,2021-05-30,https://unfccc.int/sites/default/files/NDC/202...,NaN,-24%,NaN,2025,NaN,NaN,NaN,GHG target,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,VEN,3.0,revised_first_ndc,True,2021-11-09,https://unfccc.int/sites/default/files/NDC/202...,NaN,-20%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
193,VNM,3.0,revised_first_ndc,True,2022-11-08,https://unfccc.int/sites/default/files/NDC/202...,NaN,-43.50%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
194,YEM,1.0,indc,True,2015-11-23,http://www4.unfccc.int/sites/submissions/INDC/...,NaN,-14%,NaN,2030,NaN,NaN,NaN,GHG target,Yes
195,ZMB,3.0,revised_first_ndc,True,2021-07-30,https://unfccc.int/sites/default/files/NDC/202...,-47%,NaN,2010,2030,NaN,NaN,NaN,GHG target,Yes


In [184]:
df_final['document_slug'].value_counts()

revised_first_ndc    154
first_ndc             25
second_ndc            16
indc                   2
Name: document_slug, dtype: int64

In [185]:
df_final.to_csv('../../../../data/data/CW_endpoints_data.csv', index=False)